In [38]:
import os,sys    
import re    
import requests    
from bs4 import  BeautifulSoup    
import traceback    
import pandas as pd  

In [39]:
headers={  
'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36 Core/1.77.119.400 QQBrowser/10.9.4817.400'  
} 

In [40]:
URL_PREF = 'http://172.23.254.2:40000' # 用于内容页完整URL的生成   
URL = 'http://172.23.254.2:40000/wuyou/{}' # {}内放列表页的页码 

爬取网页内容，并存储到指定文件夹，该文件夹下每个txt文件表示所要爬取的每个网页

In [41]:
def get_html(page_num):    
    """    
    获取网页html源码    
    @ page_num 指定要爬网页的页码    
    """   
    real_url = URL.format(page_num)    
    res = requests.get(real_url, headers=headers)    # 设置headers，是为了模拟成浏览器去访问这些网站  
    #res = requests.get(real_url)  
    res.encoding = 'utf-8'                           # 设置编码格式，保证内容正常显示  
    html_source = res.text                           # 获取文本一般使用res.text  
    status_code = res.status_code                    # 状态码，200（成功）表示服务器已成功处理了请求  
    return html_source,status_code                   # 返回网页源码，状态码  

In [42]:
def parse_html(keyword, total_page_number):    
    """    
    利用BeautifulSoup解析页面，获取每一个职位的内容页URL    
    所有的查询结果保存到一个list中返回。    
    @ keyword 职位中包含的关键词  
    @ total_page_number 查询列表页的总页数  
    """    
    ret_list = []                                                              # 存放职位链接  
    # try-except，可以保证程序在遇到问题时先继续运行下去  
    try:    
        for page in range(1,total_page_number+1):                             # 遍历前total_page_number个列表页  
            print ("parsing page {} of {}".format(page, total_page_number))    # 打印列表页爬取的进度  
            html_source,status_code = get_html(page)    
            if status_code == 200:                                            # 判断是否爬取成功  
                soup = BeautifulSoup(html_source, 'html.parser')              # 指定Beautiful的解析器为“html.parser”  
                t1 = soup.select('.t1 span a')                                # 在分析网页源代码的基础上，通过类名和标签进行查找  
                for i in range(1,len(t1)):    
                    job = t1[i].get('title')                                  # 获取职位    
                    if keyword not in job:                                   # 判断关键词是否存在于职位名称中  
                        continue                                             # 职位名称不符合要求，跳过当前循环  
                    href = t1[i].get('href')                                  # 获取链接    
                    ret_list.append(href)                                     # 将符合要求的内容页链接存入列表中  
        return ret_list                                                      # 返回存放职位链接的列表  
    except TimeoutError:                                                     # 连接尝试失败  
        print("请求失败")    
        return  None  

In [43]:
def save_html(save_path,ret_list):    
    """    
    利用解析到的职位的url获取职位详情页    
    """    
    cnt = 0    
    for item in ret_list:                                               # 遍历所有的职位链接  
        file_name = item.split("/")[-1]+".txt"                          # 以URL地址最后的数字来命名txt文件   
        fr = open(os.path.join(save_path,file_name),"w")                # 'w'保证生成的txt文件可以写入内容  
        cnt += 1    
        print ("geting more info for %d of %d"%(cnt, len(ret_list)))    # 打印目前获取职位详情的进度  
        try:  
            href = URL_PREF+item                                        # 根据内容页的URL规律，得到内容页完整的URL地址  
            res = requests.get(href)    
            res.encoding = 'utf-8'    
            html_source = res.text    
            fr.write(html_source)                                       # 将爬取的网页源代码写入到对应的txt文件中  
            fr.close()                                                  # 关闭打开的txt文件  
        except:    
            print(traceback.format_exc())                               # 打印详细的异常信息  
            continue                                                   # 跳过当前循环  

In [44]:
keyword = "开发"                                    # 职位的关键字，寻找包含该关键字的职位的具体信息  
save_path = "./html_source"                         # 设定职位详情保存的路径  
  
if not os.path.exists("./html_source"):            # 如果保存职位详情的文件夹不存在  
    os.mkdir("./html_source")                       # 则创建该文件夹  
  
total_page_number = 1                             # 爬取第一个列表页  
ret_list = parse_html(keyword,total_page_number)    # 批量解析列表页  
if ret_list != None:                               # 存放职位链接的列表不为空  
    save_html(save_path,ret_list)                   # 批量保存职位链接的网页源码  

parsing page 1 of 1
geting more info for 1 of 7
geting more info for 2 of 7
geting more info for 3 of 7
geting more info for 4 of 7
geting more info for 5 of 7
geting more info for 6 of 7
geting more info for 7 of 7


对上述爬取的网址进行解析

In [45]:
def get_position_detail(save_path):    
    """    
    利用解析到的职位的url获取职位和公司的详细描述，结果添加到每一职位的dict中    
    """    
    data_list = []                        # 用于存放每个职位的信息  
    cnt = 0    
    file_list = os.listdir(save_path)    # 所有网页源码的txt文件名  
    for file_name in file_list:         #　遍历所有的源代码txt文件  
        tmp_dic = {}                     # 用于存放某个职位的具体信息  
        print(file_name)    
        cnt += 1    
        print ("geting more info for %d of %d"%(cnt, len(file_list))) # 打印息息解析的进度  
        try:  
            html_path = os.path.join(save_path,file_name) # 生成完整路径  
            html_source = open(html_path).read()          # 读取网页源码  
            # print(html_source)    
              
            soup = BeautifulSoup(html_source, 'html.parser')  
              
            position_info = soup.find("div", class_="cn")                                              # 职位信息  
            title = position_info.h1["title"]                                                          # 职位名称  
            salary = position_info.strong.get_text().strip()                                           # 职位薪资  
            company_name = position_info.find("p",class_="cname").a['title']                           # 公司名称  
              
            more_info = position_info.find("p", class_="msg ltype")['title'].replace("\xa0\xa0","")  
            city = more_info.split("|")[0]                                                             # 工作地点  
            experience = more_info.split("|")[1]                                                       # 工作年限  
            recruiting_number = more_info.split("|")[2]                                                # 招聘人数  
            release_time = more_info.split("|")[3]                                                     # 发布日期  
              
            company_info = soup.find("div", class_="com_tag")  
            company_nature = company_info.findAll("p")[0]["title"]                                     # 公司性质  
            company_size = company_info.findAll("p")[1]["title"]                                       # 公司规模  
            industry = company_info.findAll("p")[2]["title"]                                           # 行业领域  
              
            descriptions = soup.find("div",class_="bmsg job_msg inbox").get_text()  
            descriptions = re.findall("\[(.*)\]",descriptions)[0]                                      # 职位信息  
            duty = soup.findAll("p", class_="fp")[0].a.get_text().strip()                              # 职能类别  
            address = soup.findAll("p", class_="fp")[1].get_text().strip().replace('上班地址：','')    # 具体地址  
              
              
            tmp_dic['job_name'] = title                                                                # 职位名称  
            tmp_dic['provide_salary'] = salary                                                         # 职位薪资  
            tmp_dic['company_name'] = company_name                                                     # 公司名称  
            tmp_dic['city'] = city                                                                     # 工作地点  
            tmp_dic['experience'] = experience                                                         # 工作年限  
            tmp_dic['recruiting_number'] = recruiting_number                                           # 招聘人数  
            tmp_dic['release_time'] = release_time                                                     # 发布时间  
  
            tmp_dic['job_info'] = descriptions                                                         # 职位信息  
            tmp_dic['job_category'] = duty                                                             # 职能类别  
            tmp_dic['company_nature'] = company_nature                                                 # 公司性质    
            tmp_dic['company_size'] = company_size                                                     # 公司规模  
            tmp_dic['industry'] = industry                                                             # 行业领域  
            tmp_dic['address'] = address                                                               # 具体地址  
              
            data_list.append(tmp_dic)    
        except:    
            print(traceback.format_exc()) # 打印异常的详细信息  
            continue                     # 跳过当前循环  
    return data_list  

In [46]:
def write_to_csv(data_list, output_file):  
    """  
    将爬取结果保存到csv文件中  
    """  
    print("Save result to %s"%output_file)  
    df = pd.DataFrame(data_list)  
    df.to_csv(output_file,index=False,sep=',')  

In [47]:
ret_positon = get_position_detail("./html_source")  
write_to_csv(ret_positon,"output.csv") 

12.txt
geting more info for 1 of 7
22.txt
geting more info for 2 of 7
24.txt
geting more info for 3 of 7
25.txt
geting more info for 4 of 7
3.txt
geting more info for 5 of 7
42.txt
geting more info for 6 of 7
5.txt
geting more info for 7 of 7
Save result to output.csv


In [53]:
data = pd.read_csv('output.csv',index_col=0) 
data

,provide_salary,company_name,city,experience,recruiting_number,release_time,job_info,job_category,company_nature,company_size,industry,address
job_name,,,,,,,,,,,,
WEB前端开发,1-1.5万/月,北京睿至大数据有限公司,北京,5-10年,招1人,2018/10/16 10:29发布,"'工作职责：', '1. 负责公司产品或客户项目的 Web 前端开发；', '2. 负责数据...",UI设计师/顾问,民营,100-499人,IT服务(系统/数据/维护),北京市朝阳区望京东园四区绿地中心（中国锦）11号楼26层
WEB前端开发,1-1.5万/月,上海冰煜信息技术工程有限公司,上海-青浦区,3-5年,招1人,2018/10/17 8:49发布,"'岗位职责：', '大型物流企业大屏可视化项目开发；', '任职要求：', '1、本科及以上...",UI设计师/顾问,民营,100-499人,计算机软件,上海市静安区新闸路831号21楼F座
数据库开发工程师,1-1.5万/月,国创恒星(合肥)软件技术有限公司,上海-浦东新区,1-3年,招1人,2018/9/25 11:00发布,"'岗位职责：', '1、负责参与SQL存储过程的编写等数据库应用开发；', '2、对数据库进...",UI设计师/顾问,股份制企业,500-999人,计算机软件,合肥市高新区长江西路669号软件园软件园2号楼
数据库开发工程师,1-1.5万/月,融慧金科金融服务外包(北京)有限公司,北京,1-3年,招1人,2018/10/12 11:30发布,"'职位亮点：BAT级别的数据量、金融科技应用。', '', '1、数据产品、工具、平台规划和...",UI设计师/顾问,民营,20-99人,互联网/电子商务,北京市朝阳区西大望路3号蓝堡国际中心1座2303
WEB前端开发,1-1.5万/月,京东商城,北京-朝阳区,3-5年,招1人,2018/9/25 11:00发布,'【公司和部门介绍】 京东（JD.com）是中国最大的自营式电商企业，2014年5月，京东在...,UI设计师/顾问,外商独资,1000-9999人,互联网/电子商务,北京市朝阳区北辰西路8号北辰世纪中心A座6层
数据库开发工程师,1-1.5万/月,泰康养老保险股份有限公司,北京-西城区,5-10年,招1人,2018/10/16 20:48发布,"'岗位职责：', '1.负责大数据相关产品的规划与设计，进行需求分析，形成产品设计以及原型构...",UI设计师/顾问,股份制企业,1000-9999人,基金/证券/期货/投资,北京市西城区复兴门内大街156号泰康人寿大厦A座
数据库开发工程师,1-1.5万/月,上海如创网络技术有限公司,上海,不限,招1人,2018/4/26 0:00发布,'因公司业务扩张，现特需大量以下岗位人才，欢迎您踊跃加入我们公司，一起努力共同成长，无经验亦...,UI设计师/顾问,民营,100-499人,通信/电信/网络设备,上海市徐汇区漕支东路81号204
